# Продвинутое машинное обучение: 
## Домашнее задание 3
Третье домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования. Как и раньше, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются. 
В этом небольшом домашнем задании мы попробуем улучшить метод Шерлока Холмса. Как известно, в рассказе The Adventure of the Dancing Men великий сыщик расшифровал загадочные письмена, которые выглядели примерно так:

Пользовался он для этого так называемым частотным методом: смотрел, какие буквы чаще встречаются в зашифрованных текстах, и пытался подставить буквы в соответствии с частотной таблицей: E — самая частая и так далее.
В этом задании мы будем разрабатывать более современный и продвинутый вариант такого частотного метода. В качестве корпусов текстов для подсчётов частот можете взять что угодно, но для удобства вот вам “Война и мир” по-русски и по-английски:
https://www.dropbox.com/s/k23enjvr3fb40o5/corpora.zip 

In [1]:
import numpy as np
import pandas as pd
import re
import random

# Task 1

1. Реализуйте базовый частотный метод по Шерлоку Холмсу:
    * подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
    * возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
    * расшифруйте их таким частотным методом.


**Решение**

Читаем данные

In [2]:
with open('./corpora/WarAndPeace.txt', 'r', encoding="utf-8") as file:
    data_ru = file.readlines()

Формируем dataframe с частотами букв и пробелов в тексте

In [3]:
regex = re.compile('\s+')
data_ru_string = regex.sub(' ', re.sub('[^а-я ]+', '', ''.join(data_ru).lower()))

In [4]:
def tokenz(text):
    letter_ru = {}
    for letter in text.lower():
        if letter in letter_ru.keys():
            letter_ru[letter] += 1
        else:
            letter_ru[letter] = 1
    return letter_ru

In [5]:
letters_dict = tokenz(data_ru_string)

letters_frame = pd.DataFrame.from_dict(letters_dict, orient='index').reset_index()
letters_frame.columns = ['letter', 'qnt']
letters_frame = letters_frame.sort_values('qnt', ascending=False)

letters_frame.head()

,letter,qnt
5,,102095
1,о,61282
4,а,45209
12,е,42519
6,и,35838


Сформируем случайное правило кодирования

In [6]:
random.seed(42)

a = ord('а')
alphabet = ' '.join([chr(i) for i in range(a,a+32)]).split() + [' ']
alphabet_mix = alphabet.copy()
random.shuffle(alphabet_mix)

coding_frame = pd.DataFrame(columns=['letter', 'code'])
coding_frame['letter'] = alphabet
coding_frame['code'] = alphabet_mix
coding_frame = coding_frame.set_index('letter')

coding_dict = coding_frame.to_dict()['code']

Закодируем предложения из Анны Карениной

In [7]:
anna_karenina_text = "Все счастливые семьи похожи друг на друга, каждая несчастливая семья несчастлива по-своему. Все смешалось в доме Облонских. Жена узнала, что муж был в связи с бывшею в их доме француженкою гувернанткой, и объявила мужу, что не может жить с ним в одном доме."
anna_karenina_text = regex.sub(' ', re.sub('[^а-я ]+', '', anna_karenina_text.lower()))
anna_karenina_text

'все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива посвоему все смешалось в доме облонских жена узнала что муж был в связи с бывшею в их доме француженкою гувернанткой и объявила мужу что не может жить с ним в одном доме'

In [8]:
anna_karenina_text_coded = ''.join([coding_dict[i] for i in anna_karenina_text])
anna_karenina_text_coded

'ламзахыаяцкльмзамй кзжшвшфкзъщнрзэызъщнрызуыфъыбзэмахыаяцклыбзамй бзэмахыаяцклызжшалшмйнзламзаймгыцша злзъшймзшецшэауквзфмэызнпэыцызхяшзйнфзеьцзлзалбпкзазеьлгмчзлзквзъшймзтщыэснфмэушчзрнлмщэыэяушозкзшедблкцызйнфнзхяшзэмзйшфмязфкя зазэкйзлзшъэшйзъшйм'

Соберем статистику на тексте из Анны Карениной

In [9]:
anna_karenina_dict = tokenz(anna_karenina_text_coded)

anna_karenina_frame = pd.DataFrame.from_dict(anna_karenina_dict, orient='index').reset_index()
anna_karenina_frame.columns = ['code', 'qnt']
anna_karenina_frame = anna_karenina_frame.sort_values('qnt', ascending=False)

anna_karenina_frame.head()

,code,qnt
3,з,42
2,м,18
13,ш,18
1,а,17
5,ы,16


Сформируем словарь для декодирования предложений

In [10]:
anna_karenina_frame['letter'] = letters_frame['letter'].values[:anna_karenina_frame.shape[0]]
temp = anna_karenina_frame[['letter', 'code']]
temp = temp.set_index('code')
decoding_dict = temp.to_dict()['letter']

anna_karenina_frame.head()

,code,qnt,letter
3,з,42,
2,м,18,о
13,ш,18,а
1,а,17,е
5,ы,16,и


Декодируем предложения

In [11]:
anna_karenina_text_decoded = ''.join([decoding_dict[i] for i in anna_karenina_text_coded])
anna_karenina_text_decoded

'нео епиекдлнчо еосял йабарл мывз ти мывзи ьирмиу тоепиекдлниу еосяу тоепиекдлни йаенаосв нео есожидаея н масо агдатеьлб роти вштиди пка свр гчд н енушл е гчнжох н лб масо юыитцвротьах звноытиткьаэ л агщунлди сврв пка то сарок рлкя е тлс н амтас масо'

# Task 2

2. Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
    * подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
    * проведите тестирование аналогично п.1, но при помощи биграмм.


**Решение**

Формируем dataframe с частотами букв и пробелов в тренировочном тексте (Война и мир)

In [12]:
def bigrams(text):
    bigrams_ru = {}
    for i in range(len(text)-1):
        bigram = text[i:i+2]
        if bigram in bigrams_ru.keys():
            bigrams_ru[bigram] += 1
        else:
            bigrams_ru[bigram] = 1
    return bigrams_ru

In [13]:
bigrams_dict = bigrams(data_ru_string)

bigrams_frame = pd.DataFrame.from_dict(bigrams_dict, orient='index').reset_index()
bigrams_frame.columns = ['bigram', 'qnt']
bigrams_frame = bigrams_frame.sort_values('qnt', ascending=False)

bigrams_frame.head()

,bigram,qnt
50,о,13217
6,и,11304
4,а,10413
56,е,9920
11,с,9798


Соберем статистику по биграмма на тексте из Анны Карениной

In [14]:
anna_karenina_bigram_dict = bigrams(anna_karenina_text_coded)

anna_karenina_bigram_frame = pd.DataFrame.from_dict(anna_karenina_bigram_dict, orient='index').reset_index()
anna_karenina_bigram_frame.columns = ['code_bigram', 'qnt']
anna_karenina_bigram_frame = anna_karenina_bigram_frame.sort_values('qnt', ascending=False)

anna_karenina_bigram_frame.head()

,code_bigram,qnt
3,за,7
2,мз,6
30,ыз,6
49,зл,5
28,зэ,5


Сформируем словарь для декодирования предложений

In [15]:
anna_karenina_bigram_frame['bigram'] = bigrams_frame['bigram'].values[:anna_karenina_bigram_frame.shape[0]]
temp = anna_karenina_bigram_frame[['bigram', 'code_bigram']]
temp = temp.set_index('code_bigram')
decoding_bigram_dict = temp.to_dict()['bigram']

anna_karenina_bigram_frame.head()

,code_bigram,qnt,bigram
3,за,7,о
2,мз,6,и
30,ыз,6,а
49,зл,5,е
28,зэ,5,с


Декодируем предложения

In [16]:
anna_karenina_text_decoded_bigram = ''.join([decoding_bigram_dict[anna_karenina_text_coded[i:i+2]] for i in range(0, len(anna_karenina_text_coded)-1, 2)])
anna_karenina_text_decoded_bigram

'оли нагора иисо сттрриим ч вилва са илваа сяру м сомь пол скно кзаноконагора иа аногю  бе  ко я дироаре  птои еррослитенал нед э да осли бсоло л оогну врелоамсье к еновя  едаы каонорнитибыойв асодк неесакема  блеелла си моалдо гльо  стее не зто пто'

# Task 3

3. Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
    * предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
    * реализуйте и протестируйте его, убедитесь, что результаты улучшились.


**Решение**

Пусть функция f : {code} -> {letter} переводит кодированный символ в букву обычного алфавита.

Тогда мы можем выбрать следующую функцию потерь: L(f) = SUMM_i log(M(f(s_i), f(s_i+1))), где 
    * s_i и s_i+1 - i и i+1 символы шифрованного текста
    * SUMM_i - суммирование по всем позициям i из зашифрованного текста
    * М - это количество биграм f(s_i) f(s_i+1) в тексте, который использовался для сбора статистики

После данных предположений алгоритм дешифровки будет состоять в следующем:
1. Выбираем случайную функцию f для дешифровки
2. Вычисляем функцию потерь L(f)
3. Формируем новую декодирующую функцию f_new перестановкой двух любых случайных символов в функции f
4. Вычисляем новую функцию потерь L(f_new)
5. Если L(f_new) >= L(f), тогда сохранем f_new
6. Если L(f_new) < L(f), тогда с веротностью L(f_new)/L(f) сохраняем f_new
7. Если f_new не сохранили то продолжаем использовать f
Повторяем пункты 2-7, пока не получим достаточно хорошо расшифрованное сообщение

Преобразуем dataframe с частотами биграмм на тренировочном тексте: выделим в отдельные столбцы первую и вторую буквы биграмм

In [17]:
bigrams_frame['first_letter'] = bigrams_frame['bigram'].apply(lambda x: x[0])
bigrams_frame['second_letter'] = bigrams_frame['bigram'].apply(lambda x: x[1])
bigrams_frame.head()

,bigram,qnt,first_letter,second_letter
50,о,13217,о,
6,и,11304,и,
4,а,10413,а,
56,е,9920,е,
11,с,9798,,с


Определим набор функций для реализации MCMC алгоритма

In [18]:
# Функция качества распознавания текста
def loss_func(text, decode_dict, bigrams_frame):
    loss = 0
    for i in range(len(text)-1):
        fc = text[i]
        sc = text[i+1]
        fl = decode_dict[fc]
        sl = decode_dict[sc]
        arr = bigrams_frame.loc[(bigrams_frame['first_letter']==fl)&(bigrams_frame['second_letter']==sl), 'qnt'].values
        if len(arr) > 0: 
            value = arr[0]
        else:
            value = 1
        loss = loss + np.log(value) 
        
    return loss

In [19]:
# Функция меняет два случайных значения словаря между собой
def make_change(decode_dict):
    new_decode_dict = decode_dict.copy()
    keys_list = random.choices(list(decode_dict), k=2)
    if keys_list[0] == keys_list[1]:
        return make_change(decode_dict)
    temp = new_decode_dict[keys_list[0]]
    new_decode_dict[keys_list[0]] = new_decode_dict[keys_list[1]]
    new_decode_dict[keys_list[1]] = temp
    return new_decode_dict

In [20]:
# Функция определяет следующий шаг
def make_step(f_new, f, dict_new, dict_):
    if f_new > f:
        return dict_new, f_new
    else:
        unif = random.uniform(0,1)
        if unif < np.exp(f_new - f):
            return dict_new, f_new
        else:
            return dict_, f

In [21]:
# Функция декодирования текста
def decoding_func(text, dec_dic):
    return ''.join([dec_dic[i] for i in text])

In [22]:
# Реализация MCMC алгоритма
def MCMC_decoding(text, decode_dict_temp, bigrams_frame, N_iter):
    f = loss_func(text=text, decode_dict=decode_dict_temp, bigrams_frame=bigrams_frame)
    for i in range(N_iter):
        new_decoding_dict = make_change(decode_dict_temp)
        new_decoding_dict = make_change(new_decoding_dict)
        while new_decoding_dict == decode_dict_temp:
            new_decoding_dict = make_change(new_decoding_dict)
        
        f_new = loss_func(text=text, decode_dict=new_decoding_dict, bigrams_frame=bigrams_frame)
        decode_dict_temp, f = make_step(f_new=f_new, f=f, dict_new=new_decoding_dict, dict_=decode_dict_temp)
        if i % 1000 == 0:
            print(f'ITER {i}: {decoding_func(text=text, dec_dic=decode_dict_temp)}')
            print(f'LOSS {i}: {round(f, 5)}')
            print()
            
    return decode_dict_temp

In [23]:
# Формируем начальное состояние словаря для декодирования, идентично первому заданию
a = ord('а')
alphabet = ' '.join([chr(i) for i in range(a,a+32)]).split() + [' ']
alphabet_mix = alphabet.copy()
random.shuffle(alphabet_mix)

decoding_frame = pd.DataFrame(columns=['letter', 'code'])
decoding_frame['code'] = alphabet
decoding_frame['letter'] = alphabet_mix
decoding_frame = decoding_frame.set_index('code')

decoding_dict_tsk3 = decoding_frame.to_dict()['letter']

In [24]:
%%time
decoding_dict_tsk3 = MCMC_decoding(text=anna_karenina_text_coded, 
                              decode_dict_temp=decoding_dict_tsk3, 
                              bigrams_frame=bigrams_frame, 
                              N_iter=10001
                             )

ITER 0: т рп ла хщэтжрп рйыэпфвзвяэпкгьопдапкгьоапуаякаъпдр ла хщэтаъп рйыъпдр ла хщэтапфв тврйьпт рп йрчащв ыптпквйрпвшщвд уэзпярдапьцдащаплхвпйьяпшжщптп тъцэп пшжтчрмптпэзпквйрпюгадбьярдувмпоьтргдадхувспэпвшнътэщапйьяьплхвпдрпйвярхпяэхып пдэйптпвкдвйпквйр
LOSS 0: 899.66682

ITER 1000: сто тхатвшлсьо торыл цеюедл буик ма буика падбан мотхатвшлсан торын мотхатвшлса цетсеори сто трожашеты с беро езшемтплю дома ичмаша хве рид зьш с тснчл т зьсжой с лю беро эуамщидомпей кисоумамвпеф л езгнслша риди хве мо редов длвы т млр с ебмер беро
LOSS 1000: 1731.08677

ITER 2000: сто тдатвмисьо торыи пехели кгуж на кгужа фалкай нотдатвмисай торый нотдатвмиса петсеору сто троцаметы с керо езментфих лона ушнама две рул зьм с тсйши т зьсцоя с их керо эганчулонфея жусогнанвфею и езбйсима рулу две но релов ливы т нир с екнер керо
LOSS 2000: 1802.52032

ITER 3000: сте ткотрмисье тевыи пабали дчуж но дчужо голдой неткотрмисой тевый неткотрмисо патсаеву сте твецоматы с даве азмантгиб лено ушномо кра вул зьм 

Итоговое расшифрованное сообщение

In [25]:
print(decoding_func(text=anna_karenina_text_coded, dec_dic=decoding_dict_tsk3))

сте тпотрмисье телыи бацади вкуг но вкуго чодвой нетпотрмисой телый нетпотрмисо батсаелу сте тлежоматы с вале азмантчиц дено ушномо пра луд зьм с тсйши т зьсжех с иц вале эконщуденчах гусекнонрчаф и азяйсимо луду пра не ладер диры т нил с авнал вале


    Видно, что текст уже начинает быть более менее похожим на оригинал. (при одном из предыдущих запусков у меня получилось достаточно хорошо приблизиться к оригиналу) 
    Ниже приведен шифр, которым было зашифровано сообщение и найденный шифр. (пробелы для наглядности)

In [26]:
print(f"Алфавит        : {' '.join(coding_dict.keys()).upper()}")
print(f"Шифр           : {' '.join([coding_dict[i] for i in coding_dict.keys()]).upper()}")
temp = list(pd.DataFrame.from_dict(decoding_dict_tsk3, orient='index').reset_index().sort_values(0)['index'])
temp = temp[1:] + [temp[0]]
print(f"Шифр найденный : {' '.join(temp).upper()}")

Алфавит        : А Б В Г Д Е Ж З И Й К Л М Н О П Р С Т У Ф Х Ц Ч Ш Щ Ъ Ы Ь Э Ю Я  
Шифр           : Ы Е Л Р Ъ М Ф П К О У Ц Й Э Ш Ж Щ А Я Н Т В С Х Г Ю Д Ь   И Ч Б З
Шифр найденный : Ш Ж Ъ Р Ф М Г Е К Б Щ Й Ц Э Ы Х Я Л А Н О Ч В У П С Ю   Ь Т И Д З


# Task 4

4. Расшифруйте сообщение:
    ←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏
    Или это (они одинаковые, второй вариант просто на случай проблем с юникодом):
    დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ


Посмотрим на количество уникальных символов в шифре

In [27]:
text_coded = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'
code_alphabet = np.unique([i for i in text_coded])
print(f'Длина алфавита: {len(code_alphabet)}')

Длина алфавита: 28


В шифре задействованы не все буквы русского алфавита, поэтому добавим в алфавит шифра 4 собственных символа, чтобы была возможность полностью расшифровать написанный текст.

In [28]:
code_alphabet = np.append(code_alphabet, ['NaN_'+str(i) for i in range(1, 6)])
print(f'Длина нового алфавита: {len(code_alphabet)}')

Длина нового алфавита: 33


Сформируем начальный словарь для декодирования

In [29]:
decoding_dict_tsk4 = {}
for i, code in enumerate(code_alphabet):
    decoding_dict_tsk4[code] = alphabet[i]
    
pd.DataFrame.from_dict(decoding_dict_tsk4, orient='index').reset_index().rename(columns={'index':'code', 0:'letter'}).head()

,code,letter
0,←,а
1,↘,б
2,↙,в
3,↝,г
4,↞,д


Применим MCMC алгоритм для расшифровки сообщения

In [30]:
%%time
decoding_dict_tsk4 = MCMC_decoding(text=text_coded, 
                                   decode_dict_temp=decoding_dict_tsk4, 
                                   bigrams_frame=bigrams_frame, 
                                   N_iter=10001
                                  )

ITER 0: ацтелкплкежезалгичйщтогпмлетелъиызелгичйщтогпмлзарцзлулхзивилциифнагедлризичпмлтаврилъчиыезщзолцричаалкцавилкплкцалцжатщтелъчщкетогилелъитуыезалйщрцейщтогпмлфщттлшщлъицтажгаалыазкачзиалшщжщгеалручцщлбиздлригаыгилдлгеыавилгалифанщс
LOSS 0: 1009.51578

ITER 1000: ешла вы вабате носпильных ала гозта носпильных текшт я жторо шоочфенау котосых лерко гсозатить шкосее вшеро вы вше шбелила гсивально а голязате пикшапильных чилл ми гошлебнее зетвестое мибинае кясши цоту конезно у назеро не очефид
LOSS 1000: 1696.49467

ITER 2000: если вы визите нодпальным или гожти нодпальным терст я этобо соочшеник ротодым лебро гдожитать сродее всебо вы все сзелали гдавильно и голяжите парсипальным чалл ха гослезнее жетведтое хазание рядса фотк ронежно к нижебо не очешащ
LOSS 2000: 1746.49005

ITER 3000: если вы визите нодпальным или гожти нодпальным терст я этобо соочшеник ротодым лебро гдожитать сродее всебо вы все сзелали гдавильно и голяжите парсипальным чалл ца гослезнее жетведтое цазание рядса х

Итоговое расшифрованное сообщение

In [31]:
print(decoding_func(text=text_coded, dec_dic=decoding_dict_tsk4))

если вы вимите норкальный или почти норкальный тедст у этого сообщения доторый легдо прочитать сдорее всего вы все смелали правильно и получите кадсикальный балл за послемнее четвертое замание дурса шотя донечно я ничего не обещаф


Текст достаточно хорошо расшифровался. По расшифровке текста уже понятно содержимое сообщения. Ниже приведен найденный шифр. (пробелы для наглядности)

In [33]:
print(f"Алфавит        : {' '.join(code_alphabet[:-5])}")
print(f"Шифр найденный : {' '.join([decoding_dict_tsk4[i] for i in code_alphabet[:]]).upper()}")

Алфавит        : ← ↘ ↙ ↝ ↞ ↟ ↤ ↨ ↲ ↳ ↷ ↹ ↾ ⇆ ⇈ ⇊ ⇌ ⇏ ⇒ ⇙ ⇛ ⇞ ⇠ ⇯ ⇰ ⇴ ⇷ ⇸
Шифр найденный : Е Ш Г Н Я И М Т Б К В   Й Щ Ь Ы Д Ф Л У О Ч С Р З А П Э Х Ю Ж Ъ Ц
